# Triggering FastAPI

`uvicorn main:app --reload`

`python -m uvicorn main.app:app --reload --port XXXX` 

# PATH Parameters


In [ ]:
# main.py

from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def root():
    return {"message": "hello world"}

In [ ]:
from fastapi import FastAPI

app = FastAPI()

# keying in localhost:port/item/3 will return a response of {"item_id":3}
@app.get("/item/{item_id}")
async def read_items(item_id: int):
    return {"item_id": item_id}

In [ ]:
from fastapi import FastAPI

app = FastAPI()

# order of path matter, if /user/me come after /user/{user_id}, respone will 
# assume receving parameter of "me"
@app.get("/user/me")
async def read_user_me():
    return {"user_id": "current user"}

@app.get("/user/{user_id}")
async def read_user(user_id: str):
    return {"user_id": user_id}

In [ ]:
from fastapi import FastAPI

app = FastAPI()

# path cannot be redefined, first path will always be returned
@app.get("/users")
async def read_user():
    return {"user": "user"}

@app.get("/users")
async def read_user2():
    return {"user": "user2"}

In [ ]:
# using a Enum class

from enum import Enum
from fastapi import FastAPI

class ModelName(str, Enum):
    alexnet = "alexnet"
    resnet = "resnet"
    lenet = "lenet"

app = FastAPI()

@app.get("/models/{model_name}")
async def get_model(model_name: ModelName):
    if model_name is ModelName.alexnet:
        return {"model_name": model_name, "message": "alexnet"}

    elif model_name.value == "lenet":
        return {"model_name": model_name, "message": "lenet"}

    else:
        return {"model_name": model_name, "message": "resnet"}

In [ ]:
# using path parameter in parameter
# required using option from Starlette

from fastapi import FastAPI

app = FastAPI()

@app.get("/files/{file_path:path}")
async def read_file(file_path: str):
    return {"file_path": file_path}

# Query Parameters

In [ ]:
from fastapi import FastAPI

app = FastAPI()

fake_item_db = [{"item_name": "foo"},{"item_name": "bar"}]

# localhost:8000/items/?skip=0&limit=10 also equate to localhost:8000/items/
@app.get("/items/")
async def read_item(skip: int=0, limit: int=10):
    return fake_item_db[skip: skip + limit]

In [ ]:
# default parameters 

from typing import Union
from fastapi import FastAPI

app = FastAPI()

@app.get("/items/{item_id}")
async def read_items(item_id: str, q: Union[str, None] = None):
    if q:
        return {"item_id": item_id, "q": q}
    return ("item_id": item_id)

In [ ]:
# Optional parameters

from fastapi import FastAPI

app = FastAPI()

@app.get("/items/{item_id}")
async def read_item(item_id: str, q:str | None = None, short: bool = False):
    item = {"item_id": item_id}
    if q:
        item.update({"q":q})
    if not short:
        item.update({"description": "long description as not_short flag is true"})
    return item

In [ ]:
# multiple path and query

from fastapi import FastAPI

app = FastAPI()

@app.get("/users/{user_id}/items/{item_id}")
async def read_user_item(user_id: int, item_id: str, q: str | None = None, short: bool = False):
    item = {"item_id": item_id}
    if q:
        item.update({"q": q})
    if not short:
        item.update({"description": "long description as not_short flag if true"})
    return item

In [ ]:
# required parameters

from fastapi import FastAPI

app = FastAPI()

@app.get("/items/{item_id}")
# needed has no default value, hence is a required field
async def read_user_item(item_id: str, needed: str):
    item = {"item_id": item_id, "needed": needed}
    return item

# Request Body

- request body: data send by client to API
- response body: data return back to client by API

In [ ]:
from typing import Union
from fastapi import FastAPI
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    description: Union[str, None] = None
    price: float
    tax: Union[float, None] = None

app = FastAPI()

@app.post("/items/")
async def create_item(item: Item):
    return item

In [ ]:
# Optional parameters

from fastapi import FastAPI
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    description: str | None = None # if default value is available, optional field
    price: float
    tax: float | None = None

app = FastAPI()

@app.post("/items/")
async def create_item(item: Item):
    return item

In [ ]:
# using the attributes of the model

from fastapi import FastAPI
from pydantic import BaseModel
from typing import Union

class Item(BaseModel):
    name: str
    description: Union[str, None] = None
    price: float
    tax: Union[float, None] = None

app = FastAPI()

@app.post("/items/")
async def create_item(item: Item):
    item_dict = item.dict()
    if item.tax:
        price_with_tax = item.price + item.tax
        item_dict.update({"price_with_tax": price_with_tax})
    return item_dict

In [ ]:
# request + path parameters

from typing import Union
from fastapi import FastAPI
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    description: Union[str, None] = None
    price: float
    tax: Union[float, None] = None

app = FastAPI()

@app.put("/items/{item_id}")
async def create_item(item_id: int, item: Item):
    return {"item_id": item_id, **item.dict()}

In [ ]:
# request + path + body parameters

from typing import Union
from fastapi import FastAPI
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    description: Union[str, None] = None
    price: float
    tax: Union[float, None] = None

app = FastAPI()

@app.put("/items/{item_id}")
async def create_item(item_id: int, item: Item, q: Union[str, None] = None):
    result = {"item_id": item_id, **item.dict()}
    if q:
        result.update({"q": q})
    return result

# Query Parameter & Strings Validation

In [ ]:
from typing import Union
from fastapi import FastAPI

app = FastAPI()

@app.get("/items/")
async def read_items(q: Union[str, None] = None):
    result = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        result.update({"q": q})
    return result

In [ ]:
# additional validation

from typing import Union
from fastapi import FastAPI, Query

app = FastAPI()

@app.get("/items/")
async def read_items(q: Union[str, None] = Query(default=None, max_length=50)):
    result = {"items":[{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        result.update({"q": q})
    return result

In [1]:
# adding more than 1 validation
from typing import Union
from fastapi import FastAPI, Query

app = FastAPI()

@app.get("/items/")
async def read_items(
    q: Union[str, None] = Query(default=None, min_length=3, max_length=50)
):
    result = {"items": [{"item_id": "Foo"},{"item_id": "Bar"}]}
    if q:
        result.update({"q": q})
    return result

In [ ]:
# Using regexp 
from typing import Union
from fastapi import FastAPI, Query

app = FastAPI()

@app.get("/items/")
async def read_items(
    q: Union[str, None] = Query(
        default=None, min_length=3, max_length=50, regex="^fixedquery$") # ^ start with $ end with
):
    result = {"items":[{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        result.update({"q": q})
    return result